In [ ]:
%load_ext autoreload
%autoreload 2

# Numpy utilities

> utilities for auxiliary numpy array processing

In [ ]:
#| default_exp data.external.numpy_utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Callable, List, Tuple
import numpy as np
import warnings

In [ ]:
#| export
warnings.filterwarnings("ignore", message="currentThread", category=DeprecationWarning)

In [ ]:
#| export
def nan_helper_1d(
    y: np.ndarray,  # utils for handling indices and logical indices of NaNs
) -> Tuple[np.ndarray, Callable]:
    """Helper to handle indices and logical indices of NaNs.

    Parameters:

        - y, 1d numpy array with possible NaNs

    Returns:

        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
            to convert logical indices of NaNs to 'equivalent' indices

    Example: linear interpolation of NaNs

        >>> nans, x= nan_helper_1d(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])

    sources:

    https://stackoverflow.com/questions/6518811/interpolate-nan-values-in-a-numpy-array
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [ ]:
#| export
def nan_interp_1d(
    y: np.ndarray,  # linear interpolation of NaNs
) -> np.ndarray:  # np array with NaNs replaced by linear interpolation
    """Linear interpolation of NaNs.

    Parameter:

        - y, 1d numpy array with possible NaNs

    Return:

        - y, 1d numpy array with NaNs replaced by linear interpolation

    Example: linear interpolation of NaNs

        >>> y = nan_interp_1d(y)

    sources:

    https://stackoverflow.com/questions/6518811/interpolate-nan-values-in-a-numpy-array
    """

    nans, x = nan_helper_1d(y)
    y[nans] = np.interp(x(nans), x(~nans), y[~nans])
    return y

In [ ]:
#| export
def ragged_nparray_list_interp(
    ragged_list_list: List[List],  # list to be interpolated with NaNs
    ob_num: int,  # number of observations
) -> np.ndarray:  # np array with NaNs replaced by linear interpolation
    """Linear interpolation of NaNs.

    Parameter:

        - y, list ragged array

    Return:

        - y, 2d numpy array with missing elements replaced by linear interpolation

    Example: linear interpolation of NaNs

        >>> y = nan_interp_1d(y)

    sources:

    https://stackoverflow.com/questions/6518811/interpolate-nan-values-in-a-numpy-array
    """

    # with np.printoptions(precision=4, suppress=True, formatter={'float': '{:0.1f}'.format}, linewidth=100):
    with np.printoptions(suppress=True, linewidth=100):
        # capture warning about ragged json arrays
        with np.testing.suppress_warnings() as sup:
            log_warning = sup.record(
                np.VisibleDeprecationWarning,
                "Creating an ndarray from ragged nested sequences",
            )
            ragged_nparray_list: np.ndarray = np.ndarray(ragged_list_list)  # type: ignore
            if len(log_warning) > 0:
                log_warning.pop()
                item_len = [len(item) for item in ragged_nparray_list]
                for count, item in enumerate(ragged_nparray_list):
                    if item_len[count] < ob_num:
                        ragged_nparray_list[count] = np.array(
                            list(ragged_nparray_list[count])
                            + [None] * (ob_num - item_len[count]),
                            dtype=np.float32,
                        )
                        ragged_nparray_list[count] = nan_interp_1d(
                            ragged_nparray_list[count]
                        )
                    elif item_len[count] > ob_num:
                        ragged_nparray_list[count] = ragged_nparray_list[count][:ob_num]
                    else:
                        pass
                aligned_nparray: np.ndarray = np.ndarray(
                    list(ragged_nparray_list), dtype=np.float32
                )
            else:
                aligned_nparray = ragged_nparray_list
    return aligned_nparray

In [ ]:
#| export
def timestamps_from_can_strings(
    can_timestamp_strings: List[str],  # list of strings from CAN
    signal_freq: float,  # signal frequency
    unit_num: int,  # number of CAN units
    unit_duration: float,  # duration in seconds per CAN unit
) -> np.ndarray:  # np array of timestamps
    """Extract timestamps from CAN strings.

    Parameters:

        - can_timestamp_strings, list of CAN strings
        - signal_freq, signal frequency
        - unit_num, number of CAN units
        - unit_duration, duration in seconds per CAN unit

    Return:

        - timestamps, 1d numpy array of timestamps

    Example: extract timestamps from "CAN" strings

        >>> timestamps_arr = timestamps_from_can_strings(can_timestamp_strings)

    """

    timestamps = []
    separators = "--T::."  # adaption separators of the raw intest string
    start_century = "20"
    for ts in can_timestamp_strings:
        # create standard iso string datetime format
        ts_substrings = [ts[i : i + 2] for i in range(0, len(ts), 2)]
        ts_iso = start_century
        for i, sep in enumerate(separators):
            ts_iso = ts_iso + ts_substrings[i] + sep
        ts_iso = ts_iso + ts_substrings[-1]
        timestamps.append(ts_iso)
    timestamps_units = (
        np.asarray(timestamps).astype("datetime64[ms]")  # in ms
        - np.timedelta64(8, "h")
    ).astype(  # convert to UTC+8
        "int"
    )  # convert to int
    if len(timestamps_units) != unit_num:
        raise ValueError(
            f"timestamps_units length is {len(timestamps_units)}, not {unit_num}"
        )
    # up-sample gears from 2Hz to 50Hz
    timestamps_seconds = list(timestamps_units)  # in list
    sampling_interval = 1.0 / signal_freq * 1000  # in ms
    unit_ob_num = int(unit_duration * signal_freq)
    timestamps = [
        i + j * sampling_interval
        for i in timestamps_seconds
        for j in np.arange(unit_ob_num)
    ]
    timestamp_arr = np.array(timestamps).reshape((unit_num, -1))

    return timestamp_arr

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()